In [ ]:
###################### augment data with flipped or rotated images ###########################
from PIL import Image
import sys
import os
import cv2
BASE_DIR = os.getcwd()

label_names = ['Happy', 'Sad', 'Fear', 'Surprise', 'Neutral', 'Angry', 'Disgust']

def aug_data():
    for label_name in label_names:
        # input_dir is the source address of the data
        input_dir = BASE_DIR + "/SFEW/" + label_name
        image_addresses = os.listdir(input_dir)
        print(len(image_addresses))

        # output_dir is the target address of the image
        output_dir = BASE_DIR + "/SFEW_aug/" + label_name
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # process each image
        for img_name in image_addresses:
            if img_name.endswith(".png"):
                img_path = input_dir + "/" + img_name
                output_path_base = output_dir + "/" + img_name[:-4]

                img = Image.open(img_path)
                output_path = output_path_base+"_original.png"
                img.save(output_path)

                img_flip = img.transpose(Image.FLIP_LEFT_RIGHT)
                output_path = output_path_base+"_flip.png"
                img_flip.save(output_path)

                img_rotate = img.transpose(Image.ROTATE_90)
                output_path = output_path_base+"_rotate_90.png"
                img_rotate.save(output_path)

                img_rotate = img.transpose(Image.ROTATE_180)
                output_path = output_path_base+"_rotate_180.png"
                img_rotate.save(output_path)

    
aug_data()        

In [ ]:
################# detect the face and crops the image #####################
import sys
import os
import cv2
import dlib

label_names = ['Happy', 'Sad', 'Fear', 'Surprise', 'Neutral', 'Angry', 'Disgust']

# use get_frontal_face_detector in the dlib
def crop_faces_from_data():
    detector = dlib.get_frontal_face_detector()

    for label_name in label_names:
        input_dir = BASE_DIR + "/SFEW/" + label_name
        image_addresses = os.listdir(input_dir)
        print(len(image_addresses))
        output_dir = BASE_DIR + "/SFEW_crop/" + label_name
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # process each image
        for img_name in image_addresses:
            if img_name.endswith(".png"):
                img_path = input_dir + "/" + img_name
                output_path = output_dir + "/" + img_name
                img = cv2.imread(img_path)
                # transfer to ray image
                gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                # use detector to get the face locations
                dets = detector(gray_img, 1)
                if len(dets) == 0:
                    cv2.imwrite(output_path, img)
                else:
                    for i, d in enumerate(dets):
                        # crop face
                        x1 = d.top() - 30 if d.top()-30 > 0 else 0
                        y1 = d.bottom() + 30 if d.bottom() > 0 else 0
                        x2 = d.left()-10 if d.left()-10 > 0 else 0
                        y2 = d.right()+10 if d.right() > 0 else 0
                        face = img[x1:y1,x2:y2]
                        # save cropped pictures
                        cv2.imwrite(output_path, face)
# crop_faces_from_data() # (optional) not used in the paper